# 🌽 Entrenamiento MobileNetV3 - ARQUITECTURA ULTIMATE V2

**Objetivo: >85% Accuracy + >80% Recall**

## 🎯 Cambios en V2:
1. ✅ **80 épocas** (aumentado de 60) para alcanzar 85%
2. ✅ **NO fine-tuning automático** (causaba colapso de 83.81% → 58%)
3. ✅ Arquitectura 384→192 (probada y funcional)
4. ✅ Batch size 32 + Cosine Decay LR
5. ⚠️ Fine-tuning SOLO SI ES ABSOLUTAMENTE NECESARIO (bloque separado)

**Resultado V1:** 83.81% en época 60 (solo faltó 1.19%)

---

## 🔧 BLOQUE 1: Setup y Verificación

In [ ]:
# 1.1 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 1.2 Clonar repositorio
!git clone -b main https://github.com/ojgonzalezz/corn-diseases-detection.git
%cd corn-diseases-detection/entrenamiento_modelos

# 1.3 Instalar dependencias
!pip install -q -r requirements.txt

# 1.4 Crear directorios necesarios en Drive
!mkdir -p /content/drive/MyDrive/corn-diseases-detection/models
!mkdir -p /content/drive/MyDrive/corn-diseases-detection/logs
!mkdir -p /content/drive/MyDrive/corn-diseases-detection/mlruns

print("\n✅ Setup completado!")

## 🏗️ BLOQUE 2: Configuración OPTIMIZADA y Modelo

In [ ]:
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers.schedules import CosineDecay
from sklearn.utils.class_weight import compute_class_weight

# Importar configuración base
from config import *
from utils import setup_gpu

# ==================== CONFIGURACIÓN ULTIMATE V2 ====================
BATCH_SIZE = 32  # Probado y funcional
EPOCHS = 80  # Aumentado de 60 para alcanzar 85%
LEARNING_RATE = 0.001  # LR inicial
EARLY_STOPPING_PATIENCE = 25  # Más paciencia (80 épocas)

# Configurar GPU
setup_gpu(GPU_MEMORY_LIMIT)

print(f"\n{'='*60}")
print("CONFIGURACIÓN ULTIMATE V2")
print(f"{'='*60}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Épocas: {EPOCHS} (aumentado de 60)")
print(f"Learning Rate: {LEARNING_RATE} (Cosine Decay)")
print(f"Early Stopping Patience: {EARLY_STOPPING_PATIENCE}")
print(f"Fine-tuning: DESHABILITADO (causaba colapso)")
print(f"{'='*60}\n")

In [ ]:
# Crear generadores de datos
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("Creando generadores de datos...\n")

# Solo rescale (augmentation ya aplicado en preprocessing)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VAL_SPLIT + TEST_SPLIT
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VAL_SPLIT + TEST_SPLIT
)

train_gen = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=RANDOM_SEED
)

val_gen = val_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=RANDOM_SEED
)

test_gen = val_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=RANDOM_SEED
)

print(f"📊 Dataset:")
print(f"  Training:   {train_gen.samples} imágenes ({train_gen.samples // BATCH_SIZE} batches)")
print(f"  Validation: {val_gen.samples} imágenes ({val_gen.samples // BATCH_SIZE} batches)")
print(f"  Test:       {test_gen.samples} imágenes ({test_gen.samples // BATCH_SIZE} batches)")

# Calcular class weights para maximizar recall
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)
class_weight_dict = dict(enumerate(class_weights))
print(f"\n⚖️ Class weights: {class_weight_dict}")

In [ ]:
# Crear modelo ULTIMATE V2
def create_ultimate_v2_model(num_classes, image_size, initial_learning_rate, steps_per_epoch, total_epochs):
    """
    Arquitectura ULTIMATE V2 - SIN fine-tuning automático
    
    Mejoras:
    - Dense(384) → Dense(192): Probada y funcional (83.81% en V1)
    - Dropout(0.4, 0.35): Regularización óptima
    - Cosine Decay ajustado a 80 épocas
    - NO fine-tuning (evita colapso catastrófico)
    """
    
    # Cargar base preentrenada
    base_model = MobileNetV3Large(
        input_shape=(*image_size, 3),
        include_top=False,
        weights='imagenet'
    )
    
    # Congelar TODAS las capas base (NO fine-tuning)
    base_model.trainable = False
    
    # ARQUITECTURA 384 → 192
    inputs = tf.keras.Input(shape=(*image_size, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    
    # Primera capa densa: 384 neuronas
    x = Dense(384, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = Dropout(0.4)(x)
    
    # Segunda capa densa: 192 neuronas
    x = Dense(192, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = Dropout(0.35)(x)
    
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    
    # Cosine Decay ajustado a 80 épocas
    lr_schedule = CosineDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=steps_per_epoch * total_epochs,
        alpha=0.1  # LR final = 10% del inicial
    )
    
    # Compilar
    model.compile(
        optimizer=Adam(learning_rate=lr_schedule),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Crear modelo
print("\n🏗️ Creando modelo ULTIMATE V2...\n")
steps_per_epoch = train_gen.samples // BATCH_SIZE

model = create_ultimate_v2_model(
    num_classes=NUM_CLASSES,
    image_size=IMAGE_SIZE,
    initial_learning_rate=LEARNING_RATE,
    steps_per_epoch=steps_per_epoch,
    total_epochs=EPOCHS
)

print(f"📐 Total parámetros: {model.count_params():,}")
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
print(f"📐 Parámetros entrenables: {trainable_params:,}")
print(f"📐 Ratio datos/params: {train_gen.samples / trainable_params:.2f}")
print("\n✅ Modelo ULTIMATE V2 creado (SIN fine-tuning)!")

## 🚀 BLOQUE 3: Entrenamiento (80 épocas)

In [ ]:
# Callbacks optimizados
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=EARLY_STOPPING_PATIENCE,
        restore_best_weights=True,
        verbose=1,
        mode='max'
    ),
    ModelCheckpoint(
        str(MODELS_DIR / 'mobilenetv3_ultimate_v2_best.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1,
        mode='max'
    )
]

print(f"\n{'='*60}")
print("🚀 INICIANDO ENTRENAMIENTO ULTIMATE V2")
print(f"{'='*60}\n")
print("Objetivo: >85% accuracy, >80% recall")
print("Arquitectura: 384→192 (V2 sin fine-tuning)")
print(f"Épocas: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: Cosine Decay desde {LEARNING_RATE}")
print(f"\nV1 alcanzó 83.81% en época 60")
print(f"V2 espera alcanzar >85% en épocas 60-80\n")
print(f"{'='*60}\n")

start_time = time.time()

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

training_time = time.time() - start_time
best_val_acc = max(history.history['val_accuracy'])
best_epoch = history.history['val_accuracy'].index(best_val_acc) + 1

print(f"\n{'='*60}")
print("✅ ENTRENAMIENTO COMPLETADO")
print(f"{'='*60}")
print(f"⏱️  Tiempo: {training_time/60:.2f} minutos")
print(f"📊 Mejor Val Accuracy: {best_val_acc:.4f} ({best_val_acc*100:.2f}%) en época {best_epoch}")
print(f"📊 Train Accuracy final: {history.history['accuracy'][-1]:.4f}")

if best_val_acc >= 0.85:
    print(f"\n🎉 ¡OBJETIVO ALCANZADO! (>85%)")
else:
    gap = (0.85 - best_val_acc) * 100
    print(f"\n⚠️  Faltaron {gap:.2f} puntos porcentuales para 85%")

print(f"{'='*60}\n")

## 📊 BLOQUE 4: Evaluación y Guardado

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import json
from datetime import datetime
from utils import evaluate_model, plot_training_history, plot_confusion_matrix, save_training_log

print(f"\n{'='*60}")
print("📊 EVALUACIÓN EN TEST SET")
print(f"{'='*60}\n")

# Evaluar modelo en test set
evaluation_results = evaluate_model(model, test_gen, CLASSES)

test_acc = evaluation_results['test_accuracy']
test_loss = evaluation_results['test_loss']

print(f"\n{'='*60}")
print("📈 RESULTADOS FINALES")
print(f"{'='*60}")
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Test Loss:     {test_loss:.4f}")

# Verificar objetivos
if test_acc >= 0.85:
    print(f"\n🎉 ¡OBJETIVO DE ACCURACY ALCANZADO! (>85%)")
else:
    print(f"\n⚠️  Accuracy por debajo del objetivo: {test_acc:.4f} < 0.85")

print(f"\n{'='*60}")
print("📋 MÉTRICAS POR CLASE")
print(f"{'='*60}")

recall_objetivo_alcanzado = True
for class_name in CLASSES:
    metrics = evaluation_results['classification_report'][class_name]
    recall = metrics['recall']
    precision = metrics['precision']
    f1 = metrics['f1-score']
    
    status = "✅" if recall >= 0.80 else "❌"
    
    print(f"\n{status} {class_name}:")
    print(f"  Precision: {precision:.4f} ({precision*100:.2f}%)")
    print(f"  Recall:    {recall:.4f} ({recall*100:.2f}%)")
    print(f"  F1-Score:  {f1:.4f} ({f1*100:.2f}%)")
    
    if recall < 0.80:
        recall_objetivo_alcanzado = False

if recall_objetivo_alcanzado:
    print(f"\n🎉 ¡OBJETIVO DE RECALL ALCANZADO EN TODAS LAS CLASES! (>80%)")
else:
    print(f"\n⚠️  Algunas clases tienen recall < 80%")

print(f"\n{'='*60}\n")

In [ ]:
# Guardar resultados
print("💾 Guardando resultados...\n")

# 1. Gráfico de entrenamiento
plot_path = LOGS_DIR / 'mobilenetv3_ultimate_v2_training_history.png'
plot_training_history(history, plot_path)
print(f"✅ Gráfico guardado: {plot_path}")

# 2. Matriz de confusión
cm_path = LOGS_DIR / 'mobilenetv3_ultimate_v2_confusion_matrix.png'
cm = plot_confusion_matrix(
    evaluation_results['y_true'],
    evaluation_results['y_pred'],
    CLASSES,
    cm_path
)
print(f"✅ Matriz de confusión guardada: {cm_path}")

# 3. Modelo final
model_path = MODELS_DIR / 'mobilenetv3_ultimate_v2_final.keras'
model.save(str(model_path))
print(f"✅ Modelo final guardado: {model_path}")

# 4. Log detallado
hyperparameters = {
    'model_name': 'MobileNetV3-Large ULTIMATE V2',
    'version': 'V2 - Sin fine-tuning',
    'architecture': 'Dense(384)->Dense(192)',
    'image_size': IMAGE_SIZE,
    'batch_size': BATCH_SIZE,
    'epochs': EPOCHS,
    'learning_rate': LEARNING_RATE,
    'lr_schedule': 'CosineDecay',
    'optimizer': 'Adam',
    'dropout': [0.4, 0.35],
    'l2_regularization': 0.001,
    'fine_tuning': 'Disabled (prevented catastrophic collapse)'
}

log_path = LOGS_DIR / 'mobilenetv3_ultimate_v2_training_log.json'

save_training_log(
    log_path,
    'MobileNetV3-Large ULTIMATE V2',
    hyperparameters,
    history,
    evaluation_results,
    cm,
    training_time
)
print(f"✅ Log guardado: {log_path}")

# 5. Resumen final
print(f"\n{'='*60}")
print("🎉 ¡ENTRENAMIENTO ULTIMATE V2 COMPLETADO!")
print(f"{'='*60}")
print(f"⏱️  Tiempo total: {training_time/60:.2f} minutos")
print(f"📊 Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"📊 Objetivo Accuracy (>85%): {'✅ ALCANZADO' if test_acc >= 0.85 else '❌ NO ALCANZADO'}")
print(f"📊 Objetivo Recall (>80%): {'✅ ALCANZADO' if recall_objetivo_alcanzado else '❌ NO ALCANZADO'}")
print(f"\n💾 Archivos guardados en:")
print(f"   • Modelo: {model_path}")
print(f"   • Logs: {LOGS_DIR}")
print(f"\n📝 Mejoras vs V1:")
print(f"   • V1: 83.81% → Colapso a 58.02% con fine-tuning")
print(f"   • V2: {test_acc*100:.2f}% SIN fine-tuning catastrófico")
print(f"{'='*60}\n")